In [2]:
import json
import openai

client = openai.Client()

In [3]:
def obter_temperatura_atual(local, unidade="celsius"):
    cidades = [
        {"local": "São Paulo", "temperatura": 32, "unidade": unidade},
        {"local": "Porto Alegra", "temperatura": 25, "unidade": unidade},
        {"local": "Rio de Janeiro", "temperatura": 38, "unidade": unidade},
    ]
    
    cidade = [cidade for cidade in cidades if cidade.get('local') == local][0]
    
    return json.dumps(cidade, ensure_ascii=False)
    
obter_temperatura_atual("São Paulo")

'{"local": "São Paulo", "temperatura": 32, "unidade": "celsius"}'

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["local"],
            },
        },
    }
]

In [5]:
funcoes_disponiveis = {
	"obter_temperatura_atual": obter_temperatura_atual
}

In [10]:
mensagens = [{'role':'user', 'content':'Qual é a temperatura em São Paulo'}]
res = client.chat.completions.create(
	model='gpt-3.5-turbo-0125',
	messages=mensagens,
	tools=tools,
	tool_choice='auto'
	
)

# res.choices[0].message.content
mensagem_res = res.choices[0].message
tool_calls = res.choices[0].message.tool_calls

In [11]:
if tool_calls:
    mensagens.append(mensagem_res)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            local=function_args.get("local"), unidade=function_args.get("unidade")
        )

        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )


In [13]:
segunda_resposta = client.chat.completions.create(
	model='gpt-3.5-turbo-0125',
	messages=mensagens
)

segunda_resposta.choices[0].message.content

'A temperatura em São Paulo é de 32 graus.'